# 🧠 BCI Competition: Motor Imagery (MI) Game Control Template

Welcome, teams\! This notebook is your template for the real-time Motor Imagery (MI) competition.

### Your Task

Your goal is to classify a user's intended "left" or "right" hand movement to control a game. You will only modify two functions: `load_model` and `predict`. The surrounding infrastructure for data recording and game communication is fixed.

1.  Add any necessary library imports in **Part 1**.
2.  Complete the `load_model` function in **Part 3**.
3.  Complete the `predict` function in **Part 4**.
4.  Submit this completed notebook and your model file(s).

## Part 1: Team-Specific Imports <span style="color:green">*(✍️ EDITABLE)*</span>.

In the cell below, add any libraries required to load your model and process the data (e.g., `tensorflow`, `sklearn`, `mne`, `scipy`).


In [1]:
# ==> YOUR CODE HERE <==
# Example:
# import mne
# from scipy.signal import butter, lfilter
# import joblib
from model.MTCformerV3 import MTCFormer
import torch
from utils.preprocessing import SignalPreprocessor
import numpy as np 
from utils.training import predict_optimized
print("Team-specific libraries would be imported here.")


Team-specific libraries would be imported here.


## Part 2: Data Recording Infrastructure <span style="color:red">*(❌ DO NOT EDIT or ADD anything)*</span>.

The system uses a dedicated function to record each trial of raw EEG data from the headset. This function captures the data and passes it to your `predict` function as a pandas DataFrame. You do not need to modify this.

```python
def record_trial(inlet):
    # This function's internal logic is fixed.
    # It records 9 seconds of data and returns a DataFrame.
    ...
```

The structure of the pandas DataFrame given to `predict()` will contain **2250 rows**, which is the total number of samples for a single 9-second trial.

```
              Time            FZ            C3           CZ  ...     Gyro3    Battery  Counter  Validation
0     1.664401e+06  332287.40625  357817.12500  640024.6250  ...  0.366211  66.666672  19733.0         1.0
1     1.664401e+06  335842.93750  361377.65625  647339.8750  ...  0.274658  66.666672  19734.0         1.0
...            ...           ...           ...          ...  ...       ...        ...      ...         ...
2248  1.664410e+06  335227.37500  358243.93750  639073.8750  ...  0.732422  66.666672  21981.0         1.0
2249  1.664410e+06  331707.93750  355082.68750  631451.1250  ...  0.823975  66.666672  21982.0         1.0
```

## Part 3: Model Loading <span style="color:green">*(✍️ EDITABLE)*</span>.

Complete this function to load your trained model from the provided file path.

**Instructions:**

  * The function takes a `model_path` string as input.
  * It must load your model and return the model object.
  * Ensure the model is in evaluation/inference mode (e.g., `model.eval()`).

In [10]:

def load_model(model_path):
    """
    Load the trained MI model from the given file path.

    Args:
        model_path (str): The path to the model file.

    Returns:
        model: The loaded model object.
    """
    model = MTCFormer(depth=3,
                    kernel_size=50,
                    modulator_kernel_size=30,
                    n_times=600,
                    chs_num=7,
                    eeg_ch_nums=4,
                    class_num=2,
                    class_num_domain=30,
                    modulator_dropout=0.48929137963218305,
                    mid_dropout=0.5,
                    output_dropout=0.42685917257840517,
                    k=100,
                    projection_dimention=2,
                    seed=4224
                    )
    model.load_state_dict(
        torch.load(model_path, weights_only=False)['model_state_dict'] , strict=False
        )
    model.eval()
    return model


## Part 4: Preprocessing and Prediction  <span style="color:green">*(✍️ EDITABLE)*</span>.

Complete this function to process the raw trial data and make a final prediction for game control.

**Instructions:**

  * The function receives the `model` object and a `df` (a pandas DataFrame) from the `record_trial` function.
  * You must perform all necessary **preprocessing** (e.g., filtering, feature extraction) inside this function.
  * The function must return one of three specific strings: `"left"`, `"right"`, or `"?"`.
  * Returning `"?"` is crucial for preventing incorrect moves in the game when the model is not confident.

In [11]:
def predict(model, df):
    """
    Preprocess the raw MI data and make a prediction.

    Args:
        model: The loaded model object.
        df (pd.DataFrame): The raw trial data.

    Returns:
        str: The prediction, which must be "left", "right", or "?".
    """
    def preprocess_optimized(
            trial_df,
            signal_processer = None
            ):
        eeg_col = ['OZ', 'PO7', 'PO8', 'PZ']


        input_array = trial_df.drop(columns = ["Time" , "Battery" , "Counter"])[eeg_col+['AccX',
            'AccY', 'AccZ', 'Gyro1', 'Gyro2', 'Gyro3' , 'Validation']].to_numpy().T
        
        acc_channel = np.linalg.norm(input_array[4:7,:],axis = 0)
        gyro_channel = np.linalg.norm(input_array[7:10,:],axis = 0)
        Validation_Channel = input_array[10,:]
        input_array[4,:] = acc_channel
        input_array[5,:] = gyro_channel
        input_array[6,:] = Validation_Channel

        input_array = input_array[:7,:]


        preprocessed_test_data , _ , _ , weights_test = signal_processer.apply_preprocessing(np.expand_dims(input_array,axis=0), np.array([None]) , np.array([1]))
        
        num_windows_per_trial = signal_processer.num_windows_per_trial

        
        return (
            torch.from_numpy(preprocessed_test_data).to(torch.float32),
           -1,
            -1,
            torch.from_numpy(weights_test).to(torch.float32),
            num_windows_per_trial
            )

    preprocessor = SignalPreprocessor(
        fs=250,                                                
        bandpass_low=6.0,                     
        bandpass_high=24.0,                  
        n_cols_to_filter=4,                   
        window_size=600,                      
        window_stride=35,                    
        idx_to_ignore_normalization=-1,        
        crop_range=(2.5 , 7)              
    )

    data , _ , _ , weights , windows_per_trial = preprocess_optimized(df,signal_processer=preprocessor)

    
    mapping_mi = {
        0:"left",
        1:"right"
    }

    prediction = predict_optimized(
        model=model,
        windows_per_trial=windows_per_trial,
        loader= (data , weights)

    )

    return mapping_mi[prediction[0]]

## Part 5: Game Interface <span style="color:red">*(❌ DO NOT EDIT or ADD anything)*</span>.

After your `predict` function returns a command (e.g., `"left"`), the system passes this string to a game interface function. This function sends your prediction over the network to the game engine.

```python
def send_to_game(socket, prediction):
    # This function's internal logic is fixed.
    # It sends your prediction string ("left", "right", or "?")
    # over a ZeroMQ socket to the game.
    ...